In [2]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re
import random
random.seed(10)

import os

# Global Path variables
source_path = "/media/andres/T7 Shield/ucan_lymfom"
slicer_path = "/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"
# 'C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'
data_path = os.path.join(source_path, 'Final_Selected_exams_from_U-CAN-Lymfom.xlsx')
destination_path = os.path.join(source_path, 'Selected_for_UCAN_project/Resampled_Selected_Sample_Images')



In [3]:
%env SITK_SHOW_COMMAND "/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

env: SITK_SHOW_COMMAND="/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"


In [5]:
master_data = pd.read_excel(data_path)
print(master_data.shape)
master_data.head()

(2264, 8)


,Unnamed: 0,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
0,0,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr900031979078-20220919/PT-20220919-141702-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr900031979078-20220919,PT-20220919-141702-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr900031979078,20220919
1,1,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr900031979078-20220919/CT-20220919-143055-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr900031979078-20220919,CT-20220919-143055-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,npr900031979078,20220919
2,2,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr748236341307-20220908/PT-20220908-103801-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr748236341307-20220908,PT-20220908-103801-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr748236341307,20220908
3,3,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr748236341307-20220908/CT-20220908-105049-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr748236341307-20220908,CT-20220908-105049-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,npr748236341307,20220908
4,4,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr652851209495-20220829/PT-20220829-124104-3.8_WB_F18_FDG_VENFAS-QCFX-S_400_M.Free-2.8000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr652851209495-20220829,PT-20220829-124104-3.8_WB_F18_FDG_VENFAS-QCFX-S_400_M.Free-2.8000mm,ASPTCTX0001,npr652851209495,20220829


In [6]:
patdir_list = list(set(master_data['patient_directory'].to_list()))

In [7]:
sampledir = random.sample(patdir_list,5)
print(len(sampledir))
print(sampledir)

5
['ASPTCTX0001_npr406981575115-20220325', 'ASPTCTX0001_npr476083448226-20170411', 'ASPTCTX0001_npr485030831082-20220517', 'ASPTCTX0001_npr136457009354-20191223', 'ASPTCTX0001_npr588137148278-20150303']


In [9]:
sample_df = master_data[master_data.patient_directory.isin(sampledir)].copy()
print(sample_df.shape)
sample_df[sample_df['PET-CT_info'].str.contains('CT')].shape

(10, 8)


(7, 8)

In [10]:
sample_df['num_slices'].to_list()

KeyError: 'num_slices'

In [16]:
r"""
Skipped_images:
F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm

distorted : npr648969352384_CT-20160318-150152-6.6-UAS-WB-FDG-3D-VENFASTHORAX-INSP-20160120-WB-VEN-AX-0.964844mm_.nii.gz
"""

'\nSkipped_images:\nF:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\\ASPTCTX0001_npr252097509348-20180509\\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm\n'

In [11]:
new_size = [356, 356, 356]
new_spacing = [2, 2, 3]

In [13]:
for index, row in sample_df.iterrows():

    #vol_img = sitk.ReadImage(row['directory'])
    
    print(row['directory'])

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
    reader.SetFileNames(dicom_names)
    vol_img = reader.Execute()
        
    #res_img = resample_volume(vol_img, sitk.sitkLinear, new_size, new_spacing)
    
    res_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                         vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                         vol_img.GetPixelID())

    sitk.WriteImage(image = res_img, fileName = destination_path + '_' +
                    str(row['npr']) + '_' + str(row['PET-CT_info']) + '_.nii.gz')



/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr485030831082-20220517/PT-20220517-144331-3.8_WB_F18_FDG_VENFAS-QCFX-S_400_M.Free-2.8000mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr485030831082-20220517/CT-20220517-145542-3.8_WB_F18_FDG_VENFAS-WB_Venfas_3_ax-3.000000mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr406981575115-20220325/PT-20220325-090400-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-QCFX-S_400_M.Free-2.8000mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr406981575115-20220325/CT-20220325-091723-3.11_WB_F18_FDG_VENFAS__THORAX_INANDAD-WB_Venfas_3_ax-3.000000mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr136457009354-20191223/PT-20191223-102800-3.7_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr136457009354-20191223/CT-20191223-104136-3.7_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm
/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr476083448226-20170411/PT-20170411-115946-6.39_WB_FDG_CT_NATIV-WB_AC_FDG_VPFX-S-2.

In [23]:
if not os.path.exists(destination_path):
        os.makedirs(destination_path)

In [14]:
reader = sitk.ImageSeriesReader()
#dicom_names = reader.GetGDCMSeriesFileNames(r'F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm')
dicom_names = reader.GetGDCMSeriesFileNames('/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_npr207978513481-20190903/CT-20190903-122850-3.6_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm')
reader.SetFileNames(dicom_names)
vol_img = reader.Execute()

res_img = sitk.Resample(vol_img, new_size, sitk.Transform(), sitk.sitkLinear,
                        vol_img.GetOrigin(), new_spacing, vol_img.GetDirection(), 0,
                        vol_img.GetPixelID())

# sitk.WriteImage(image = res_img, fileName = r'D:\ucan_lymfom\Selected_for_UCAN_project\Resampled_Selected_SampleImages\\CT_sample3Dimg_.nii.gz')


ImageSeriesReader (0x4264110): Non uniform sampling or missing slices detected,  maximum nonuniformity:914.997



In [16]:
print(vol_img.GetSize())
print(res_img.GetSize())

(512, 512, 762)
(356, 356, 356)


In [32]:
dimg = sitk.GetArrayFromImage(vol_img)
print(dimg.max())
print(dimg.min())

3071
-1024


In [50]:
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(r'F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr462153436708-20220302\PT-20220302-130601-3.8-WB-F18-FDG-VENFAS-QCFX-S-400-M.Free-2.8000mm_JK')
reader.SetFileNames(dicom_names)
vol_img = reader.Execute()
dimg = sitk.GetArrayFromImage(vol_img)
print(dimg.max())
print(dimg.min())

34410.92039
0.0


In [6]:
dicom_names = r'F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm'
master_data[master_data['directory'] == dicom_names]

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
194,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm,ASPTCTX0001_npr252097509348-20180509,CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm,1.2.840.113619.2.5.1207072.12420.1525686926.934,npr252097509348,74.0,'77.0','1.64',512,512,1151,1.571045,1.571045,0.976562,"(512, 512, 1151)","(1.571045, 1.571045, 0.976562)","[-0.488281, -402.187500, 121.687500]","[0.000000, 1.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.620.1525680322.102.12550.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED']",'20180509','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE'


In [7]:
dicom_names1 = r'F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm'
master_data[master_data['directory'] == dicom_names1]

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model


In [10]:
metadata = pd.read_excel(r'F:\U-CAN-Lymfom_A\Ashish_SampleResampling\metadata_extraction\metadata_extract_29Sep2023.xlsx')
metadata.head(1)

,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
0,F:/U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr258071002437-20160805\CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,2291,888,1,0.545455,0.596847,1250.090942,"[0.545455, 0.596847]","(2291, 888, 1)","(0.545455, 0.596847, 1250.090942)","[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE'


In [15]:
img = 'CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm'
img2 = 'CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm'
metadata[np.logical_or(metadata['image_dir'] == img2, metadata['image_dir'] == img)]

,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
4641,F:/U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr252097509348-20180509,CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm,1.2.840.113619.2.5.1207072.12420.1525686926.935,'ASPTCTX0001_npr252097509348','074Y','O','77.0','1.64',512,512,512,512,1151,0.976562,0.976562,0.949219,"[0.976562, 0.976562]","(512, 512, 1151)","(0.976562, 0.976562, 0.949219)","[-250.488281, -229.980469, 121.962494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.620.1525680322.102.12550.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20180509','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE'
4642,F:/U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr252097509348-20180509\CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr252097509348-20180509,CT-20180509-103313-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm,1.2.840.113619.2.5.1207072.12420.1525686926.934,'ASPTCTX0001_npr252097509348','074Y','O','77.0','1.64',512,512,512,512,1151,1.571045,1.571045,0.976562,"[1.571045, 1.571045]","(512, 512, 1151)","(1.571045, 1.571045, 0.976562)","[-0.488281, -402.187500, 121.687500]","[0.000000, 1.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.620.1525680322.102.12550.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED']",'20180509','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE'


In [38]:
patientgrp = master_data.groupby(['patient_directory']).agg({'dicom_img':'count'}).reset_index()
patientgrp[patientgrp['dicom_img']!=2]

,patient_directory,dicom_img
34,ASPTCTX0001_npr150867785519-20131210,1
42,ASPTCTX0001_npr167850885413-20160620,3
73,ASPTCTX0001_npr218702301637-20161020,1
101,ASPTCTX0001_npr252097509348-20170725,1
140,ASPTCTX0001_npr284872448692-20160921,1
143,ASPTCTX0001_npr290047476872-20161220,1
153,ASPTCTX0001_npr292781072009-20180409,1
169,ASPTCTX0001_npr314033739598-20151215,1
192,ASPTCTX0001_npr338511978351-20170201,1
193,ASPTCTX0001_npr338511978351-20170531,1


In [40]:
master_data[master_data['patient_directory']=='ASPTCTX0001_npr877253951206-20181029']

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
871,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,ASPTCTX0001_npr877253951206-20181029,CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,1.2.840.113619.2.405.3.2831160321.842.1540794498.931.1,npr877253951206,75.0,'50.0','1.58',512,512,1283,0.976562,0.976562,0.625,"(512, 512, 1283)","(0.976562, 0.976562, 0.625)","[-250.000, -250.000, 6.810]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20181029','PET FDG WB 1 h','WB Venfas 0.6 ax','6.39 WB_FDG CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
872,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,ASPTCTX0001_npr877253951206-20181029,CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,1.2.840.113619.2.5.4250371947.12760.1540806994.836,npr877253951206,75.0,'50.0','1.58',512,512,322,0.976562,0.976562,3.000,"(512, 512, 322)","(0.976562, 0.976562, 3.0)","[-241.699, -213.378, 6.184]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20181029','PET FDG WB 1 h','WB Venfas 3 ax','6.39 WB_FDG CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
873,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,ASPTCTX0001_npr877253951206-20181029,PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,1.2.840.113619.2.405.2831160321.1540817816.268514,npr877253951206,75.0,'50.0','1.58',256,256,287,1.953125,1.953125,2.790,"(256, 256, 287)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -792.69000244140]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20181029','PET FDG WB 1 h','QCFX-S 400','6.39 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",'PT','GE MEDICAL SYSTEMS','Discovery MI'


In [41]:
master_data[master_data['patient_directory']=='ASPTCTX0001_npr877253951206-20181029']

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
871,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,ASPTCTX0001_npr877253951206-20181029,CT-20181029-140238-6.39-WB_FDG-CT-VENFAS-WB-Venfas-0.6-ax-0.625000mm,1.2.840.113619.2.405.3.2831160321.842.1540794498.931.1,npr877253951206,75.0,'50.0','1.58',512,512,1283,0.976562,0.976562,0.625,"(512, 512, 1283)","(0.976562, 0.976562, 0.625)","[-250.000, -250.000, 6.810]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20181029','PET FDG WB 1 h','WB Venfas 0.6 ax','6.39 WB_FDG CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
872,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,ASPTCTX0001_npr877253951206-20181029,CT-20181030-072512-6.39-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm,1.2.840.113619.2.5.4250371947.12760.1540806994.836,npr877253951206,75.0,'50.0','1.58',512,512,322,0.976562,0.976562,3.000,"(512, 512, 322)","(0.976562, 0.976562, 3.0)","[-241.699, -213.378, 6.184]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20181029','PET FDG WB 1 h','WB Venfas 3 ax','6.39 WB_FDG CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery MI'
873,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr877253951206-20181029\PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr877253951206-20181029\PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,ASPTCTX0001_npr877253951206-20181029,PT-20181029-135206-6.39-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm,1.2.840.113619.2.405.2831160321.1540817816.268514,npr877253951206,75.0,'50.0','1.58',256,256,287,1.953125,1.953125,2.790,"(256, 256, 287)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -792.69000244140]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.2831160321.842.1540794498.408.7450.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20181029','PET FDG WB 1 h','QCFX-S 400','6.39 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",'PT','GE MEDICAL SYSTEMS','Discovery MI'


In [43]:
master_data[master_data['patient_directory']=='ASPTCTX0001_npr391123015354-20151125']

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
363,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\ASPTCTX0001_npr391123015354-20151125\PT-20151125-105902-UAS-WB-FDG-3D-CT-THORAX-BUK-VENFAS-20141209-WB-FDG-AC-3.2700mm,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr391123015354-20151125\PT-20151125-105902-UAS-WB-FDG-3D-CT-THORAX-BUK-VENFAS-20141209-WB-FDG-AC-3.2700mm,ASPTCTX0001_npr391123015354-20151125,PT-20151125-105902-UAS-WB-FDG-3D-CT-THORAX-BUK-VENFAS-20141209-WB-FDG-AC-3.2700mm,1.2.840.113619.2.131.4240671514.1448445893.483663,npr391123015354,71.0,'55.0','1.54',128,128,247,3.90625,3.90625,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, -671.90002441406]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.949.1447163216.740.12830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20151125','FDG WB 1 h','WB-FDG-AC','UAS-WB-FDG-3D CT THORAX-BUK VENFAS 20141209',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",'PT','GE MEDICAL SYSTEMS','Discovery STE'


In [45]:
master_data.patient_id.nunique()

481

In [46]:
master_data.patient_directory.nunique()

705

In [47]:
patientgrp[patientgrp['dicom_img']==2].shape

(671, 2)